In [17]:
import pennylane as qml
import numpy as np
from scipy.sparse.linalg import eigsh
L_x = 2
L_y = 2

t = 1
U = 8

# This is the characteristic of the polynomial filter with cos(x)^50, which we later use
optimal_x_for_filter = 0.02

H_penny = qml.spin.fermi_hubbard('rectangle',[L_x, L_y], hopping=t, coulomb=U)

H_original = H_penny.sparse_matrix().toarray()
print(f"Numpy Ground state energy: {np.min(np.linalg.eigvalsh(H_original))}")

eig_vals, eig_vecs = eigsh(H_penny.sparse_matrix(), k=6, which='SA')
print(f"PennyLane Ground state energy: {np.min(eig_vals)}")

H_matrix = H_penny.sparse_matrix()
eig_max = eigsh(H_matrix, k=1, which='LA', return_eigenvectors=False)[0]
eig_min = np.min(eig_vals)
print(f"Max eigenvalue: {eig_max}")
print(f"Min eigenvalue: {eig_min}")

print("Shifting Hamiltonian...")
ops = H_penny.operands
coefs = [op.scalar for op in ops]
obs = [op.base for op in ops]
H_penny = qml.Hamiltonian(coefs, obs)

H_penny = (H_penny - qml.Identity(wires=H_penny.wires) * (eig_min - optimal_x_for_filter * (eig_max - eig_min))) / (eig_max - eig_min)
H_penny = qml.Hamiltonian(H_penny.coeffs, H_penny.ops)

H_matrix = H_penny.sparse_matrix()
eig_max = eigsh(H_matrix, k=1, which='LA', return_eigenvectors=False)[0]
eig_min = eigsh(H_matrix, k=1, which='SA', return_eigenvectors=False)[0]
print(f"Max eigenvalue: {eig_max}")
print(f"Min eigenvalue: {eig_min}")

Numpy Ground state energy: -3.207750943219351
PennyLane Ground state energy: -3.207750943219336
Max eigenvalue: 32.000000000000064
Min eigenvalue: -3.207750943219336
Shifting Hamiltonian...
Max eigenvalue: 1.019999999999999
Min eigenvalue: 0.01999999999999943


In [18]:
H_penny

(
    -0.01420141834127278 * (Y(0) @ Z(1) @ Y(2))
  + -0.01420141834127278 * (X(0) @ Z(1) @ X(2))
  + 0.22722269346036447 * I([0, 1, 2, 3, 4, 5, 6, 7])
  + -0.01420141834127278 * (Y(1) @ Z(2) @ Y(3))
  + -0.01420141834127278 * (X(1) @ Z(2) @ X(3))
  + -0.01420141834127278 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4))
  + -0.01420141834127278 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4))
  + -0.01420141834127278 * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5))
  + -0.01420141834127278 * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5))
  + -0.01420141834127278 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + -0.01420141834127278 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + -0.01420141834127278 * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + -0.01420141834127278 * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + -0.01420141834127278 * (Y(4) @ Z(5) @ Y(6))
  + -0.01420141834127278 * (X(4) @ Z(5) @ X(6))
  + -0.01420141834127278 * (Y(5) @ Z(6) @ Y(7))
  + -0.01420141834127278 * (X(5) @ Z(6) @ X(7))
  + -0.05680567336509112 * Z(1)
  + -0.05680567336509112 * Z(0)


In [19]:
ops = H_penny.operands
coefs = [op.scalar for op in ops]
print(f"Number of terms: {len(ops)}")
print(f"Coefficients: {coefs[:5]}...")  # Show first 5
print(f"Operators: {ops[:5]}...")  # Show first 5

Number of terms: 30
Coefficients: [np.float64(-0.01420141834127278), np.float64(-0.01420141834127278), np.float64(0.22722269346036447), np.float64(-0.01420141834127278), np.float64(-0.01420141834127278)]...
Operators: (-0.01420141834127278 * (Y(0) @ Z(1) @ Y(2)), -0.01420141834127278 * (X(0) @ Z(1) @ X(2)), 0.22722269346036447 * I([0, 1, 2, 3, 4, 5, 6, 7]), -0.01420141834127278 * (Y(1) @ Z(2) @ Y(3)), -0.01420141834127278 * (X(1) @ Z(2) @ X(3)))...


In [20]:
obs = [op.base for op in ops]
grouped = qml.pauli.group_observables(obs, coefs, grouping_type='commuting')
print(f"Number of groups: {len(grouped)}")
print(f"Shape of first group: {len(grouped[0])}")
print(f"Shape of second group: {len(grouped[1])}")
for g in grouped[0]:
    print(len(g))
for g in grouped[1]:
    print(len(g))
    
obs_flat = [item for sublist in grouped[0] for item in sublist]
coeffs_flat = [item for sublist in grouped[1] for item in sublist]

grouped_H = qml.Hamiltonian(coeffs_flat, obs_flat)
eig_vals_grouped, _ = eigsh(grouped_H.sparse_matrix(), k=6, which='SA')
print(f"Check Ground state energy: {np.min(eig_vals_grouped)}")

Number of groups: 2
Shape of first group: 3
Shape of second group: 3
8
14
8
8
14
8
Check Ground state energy: 0.019999999999999404


In [21]:
dev = qml.device("default.qubit", wires=H_penny.num_wires)
@qml.qnode(dev)
def my_circ(time, n_steps=1, order=2):
    # Prepare some state
    for w in range(H_penny.num_wires):
        qml.Hadamard(w)
    
    # Evolve according to H
    qml.TrotterProduct(grouped_H, time=time, n=n_steps, order=order, check_hermitian=True)

    # Measure some quantity
    return qml.state()

In [22]:
from scipy.linalg import expm


time = 1.0
order = 2
n_steps = 10

psi_trotter = my_circ(time, n_steps, order)

# Compute expectation value using exact matrix exponential

H_matrix = grouped_H.sparse_matrix().toarray()


# Exact time evolution
U_exact = expm(1j * time * H_matrix)

# Initial state (same as in circuit: all Hadamards)
psi_0 = np.ones(2**grouped_H.num_wires) / np.sqrt(2**grouped_H.num_wires)

# Apply exact evolution
psi_exact = U_exact @ psi_0

print(f"Exact state (first 10 elements): {psi_exact[:10]}")
print(f"Trotter state (first 10 elements): {psi_trotter[:10]}")
print(f"Fidelity: {abs(np.vdot(psi_exact, psi_trotter))**2}")

Exact state (first 10 elements): [0.06211461+0.00693005j 0.06240787+0.0033923j  0.06240787+0.0033923j
 0.05876258+0.02069239j 0.06240787+0.0033923j  0.06201442+0.00691887j
 0.06122646+0.01036504j 0.05886069+0.02071483j 0.06240787+0.0033923j
 0.06280237+0.0034727j ]
Trotter state (first 10 elements): [0.06211461+0.00693005j 0.06240787+0.0033923j  0.06240787+0.0033923j
 0.05876259+0.02069236j 0.06240787+0.0033923j  0.06201442+0.00691887j
 0.0612265 +0.01036489j 0.05886072+0.02071474j 0.06240787+0.0033923j
 0.06280233+0.00347285j]
Fidelity: 0.9999999999973412


In [23]:
# This is what I wanted to implement for clean reading, but it explodes in memory usage due to @
# 
# import numpy as np
# from scipy.optimize import minimize

# d = 10  # Grad der QSP-Schleifen

# # Zielpolynom: P(cos(x))
# def P(x):
#     return 0.0 + 1.0*np.cos(x)**(d-1)


# def R(theta, phi=0, gamma=0):
#     # return qml.matrix(qml.RY(theta, wires=0)) #  @ qml.RZ(np.pi, wires=0))
#     return np.array([[np.cos(theta/2), -np.sin(theta/2)],
#                     [np.sin(theta/2), np.cos(theta/2)]])
#     return np.array([
#         [np.exp(1j*(gamma+phi))*np.cos(theta), np.exp(1j*phi)*np.sin(theta)],
#         [np.exp(1j*gamma)*np.sin(theta), -np.cos(theta)]
#     ])

# def U_signal(x, which=0):
#     """Signal: wirkt auf |0> oder |1> Komponente"""
#     if which==0:
#         return np.diag([np.exp(1j*x), 1])
#     else:
#         return np.diag([1, np.exp(-1j*x)])
# def qsp_matrix(angles, x):
#     d = len(angles)//2
#     theta0 = angles[0]
#     theta1 = angles[1:d+1]
#     theta2 = angles[d+1:2*d+1]
    
#     # Start-Rotation
#     M = R(theta0)
    
#     # Erste Schleife: wirkt auf |0> Komponente
#     for t in theta1:
#         M = R(t) @ U_signal(x, 0) @ M
        
#     # Zweite Schleife: wirkt auf |1> Komponente
#     for t in theta2:
#         M = R(t) @ U_signal(x, 1) @ M
    
#     return M

# xs = np.linspace(0, np.pi, 20)
# def loss(angles):
#     L = 0
#     for x in xs:
#         M = qsp_matrix(angles, x)
#         amp = M[0,0]  # |0> Amplitude
#         L += np.abs(amp - P(x))**2
#     return L
# np.random.seed(42)
# init_angles = np.random.uniform(0, np.pi, 2*d+1)  # zwei Sets

# res = minimize(loss, init_angles, method='L-BFGS-B', options={'maxiter':500})
# opt_angles = res.x
# print("Optimierte Winkel:", opt_angles)

# x_test = 0.5
# M = qsp_matrix(opt_angles, x_test)
# print("Amplitude |0>:", M[0,0])
# print("Ziel P(cos(x)):", P(x_test))

In [24]:
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

d = 5000

def P(x):
    return jnp.cos(x)**(d-2)

def R(theta):
    return jnp.array([
        [jnp.cos(theta/2), -jnp.sin(theta/2)],
        [jnp.sin(theta/2),  jnp.cos(theta/2)]
    ], dtype=jnp.complex128)

def U_signal(x, which):
    if which == 0:
        return jnp.array([[jnp.exp(1j*x), 0.0],
                          [0.0, 1.0]], dtype=jnp.complex128)
    else:
        return jnp.array([[1.0, 0.0],
                          [0.0, jnp.exp(-1j*x)]], dtype=jnp.complex128)

from jax import lax

@jax.jit
def qsp_matrix2(angles, x):
    d = (angles.shape[0] - 1) // 2

    theta0 = angles[0]
    theta1 = angles[1:1+d]
    theta2 = angles[1+d:1+2*d]

    M = jnp.array([
        [jnp.cos(theta0/2), -jnp.sin(theta0/2)],
        [jnp.sin(theta0/2),  jnp.cos(theta0/2)]
    ], dtype=jnp.complex128)

    def loop0(M, theta):
        M = jnp.array([
        [jnp.cos(theta/2), -jnp.sin(theta/2)],
        [jnp.sin(theta/2),  jnp.cos(theta/2)]
    ], dtype=jnp.complex128) @ jnp.array([[jnp.exp(1j*x), 0.0],
                          [0.0, 1.0]], dtype=jnp.complex128) @ M
        return M, None

    def loop1(M, theta):
        M = jnp.array([
        [jnp.cos(theta/2), -jnp.sin(theta/2)],
        [jnp.sin(theta/2),  jnp.cos(theta/2)]
    ], dtype=jnp.complex128) @ jnp.array([[1.0, 0.0],
                          [0.0, jnp.exp(-1j*x)]], dtype=jnp.complex128) @ M
        return M, None

    M, _ = lax.scan(loop0, M, theta1)
    M, _ = lax.scan(loop1, M, theta2)

    return M

@jax.jit
def loss(angles):
    xs = jnp.linspace(0.0, 1.5, d * 7 // 11)

    def loss_x(x):
        M = qsp_matrix2(angles, x)
        amp = M[0,0]
        return jnp.abs(amp - P(x))**2

    return jnp.sum(jax.vmap(loss_x)(xs))

loss_grad = jax.jit(jax.grad(loss))



In [25]:
import numpy as np
from scipy.optimize import minimize

loss_np = lambda a: np.array(loss(jnp.array(a)), dtype=float)

init_angles = np.random.uniform(0, np.pi, 2*d+1)

res = minimize(loss, init_angles, method="BFGS", options={"maxiter":500}, jac=loss_grad)
opt_angles = res.x
print("Optimierte Winkel:", opt_angles)


Optimierte Winkel: [1.17716251 1.99559319 0.90675127 ... 2.42885153 1.59935697 0.23934347]


In [26]:
res

  message: Maximum number of iterations has been exceeded.
  success: False
   status: 1
      fun: 0.0007616051906933926
        x: [ 1.177e+00  1.996e+00 ...  1.599e+00  2.393e-01]
      nit: 500
      jac: [ 2.555e-02  1.778e-02 ...  9.192e-04 -4.200e-03]
 hess_inv: [[ 9.514e-01 -3.370e-02 ...  3.119e-03  3.449e-03]
            [-3.370e-02  9.655e-01 ... -4.182e-03 -7.596e-03]
            ...
            [ 3.119e-03 -4.182e-03 ...  9.216e-01 -3.422e-02]
            [ 3.449e-03 -7.596e-03 ... -3.422e-02  9.448e-01]]
     nfev: 506
     njev: 506

In [27]:
x_test = 0.4
M = qsp_matrix2(opt_angles, x_test)
print("Amplitude |0>:", M[0,0])
print("Ziel P(cos(x)):", P(x_test))

Amplitude |0>: (-4.1068224686573174e-05+0.00032945855810076474j)
Ziel P(cos(x)): 3.2611970094831076e-179


In [28]:
# Remap wires of grouped_H to make wire 0 free for control
def our_R(theta, wire):
    qml.RY(theta, wires=wire) # @ qml.RZ(np.pi, wires=wire)


wire_map = {i: i+1 for i in range(grouped_H.num_wires)}
grouped_H_new = qml.map_wires(grouped_H, wire_map)


def iterate(angles):
    d = len(angles)//2
    theta0 = angles[0]
    theta1 = angles[1:d+1]
    theta2 = angles[d+1:2*d+1]
    
    for _ in range(1):
        our_R(theta0, wire=0)
        
        # Erste Schleife: wirkt auf |0> Komponente
        for t in theta1:
            qml.ctrl(qml.TrotterProduct(grouped_H_new, time=1.0, n=n_steps, order=order, check_hermitian=True), control=0, control_values=0)
            our_R(t, wire=0)
            
        # Zweite Schleife: wirkt auf |1> Komponente
        for t in theta2:
            qml.ctrl(qml.TrotterProduct(grouped_H_new, time=-1.0, n=n_steps, order=order, check_hermitian=True), control=0, control_values=1)
            our_R(t, wire=0)



dev = qml.device("default.qubit", wires=grouped_H.num_wires + 1)
@qml.qnode(dev)
def my_circ_ctrl(time, n_steps=1, order=2, angles=None):
    # Prepare some state
    for w in range(1, grouped_H.num_wires + 1):
        qml.Hadamard(w)
    iterate(angles)
        
    # Measure some quantity
    return qml.state()     


In [29]:
rr = my_circ_ctrl(time, n_steps, order, opt_angles)

In [30]:
r_now = psi_0  # rr.reshape(-1,2)[:,0]
energy = np.vdot(r_now, H_penny.sparse_matrix().toarray() @ r_now).real / np.vdot(r_now, r_now).real
print(f"Erwartungswert des Hamiltonians: {energy}")
r_now = rr.reshape(2,-1)[0]
energy = np.vdot(r_now, H_penny.sparse_matrix().toarray() @ r_now).real / np.vdot(r_now, r_now).real
print(f"Erwartungswert des Hamiltonians: {energy}")
energy = np.vdot(r_now, H_original @ r_now).real / np.vdot(r_now, r_now).real
print(f"Erwartungswert des originalen Hamiltonians: {energy}")
print(f"Norm r_now: {np.vdot(r_now, r_now).real}, Norm rr: {np.vdot(rr, rr).real}")

Erwartungswert des Hamiltonians: 0.33833191961890474
Erwartungswert des Hamiltonians: 0.021204543488649138
Erwartungswert des originalen Hamiltonians: -3.165341676070699
Norm r_now: 7.365308161254557e-05, Norm rr: 0.9999999999198295


### The following can not be implemented without post selection after every iteration, so it is just to check if the algorithm would work

In [31]:
psi = psi_0

@qml.qnode(dev)
def my_one_iter(time, n_steps=1, order=2, angles=None):
    # Prepare some state
    qml.StatePrep(psi, wires=range(1, grouped_H.num_wires + 1))
    iterate(angles)
        
    # Measure some quantity
    return qml.state()    

In [32]:
rr2 = my_one_iter(time, n_steps, order, opt_angles)

In [33]:
r_now = rr2.reshape(2,-1)[0]
energy = np.vdot(r_now, H_penny.sparse_matrix().toarray() @ r_now).real / np.vdot(r_now, r_now).real
print(f"Erwartungswert des Hamiltonians: {energy}")
energy = np.vdot(r_now, H_original @ r_now).real / np.vdot(r_now, r_now).real
print(f"Erwartungswert des originalen Hamiltonians: {energy}")
print(f"Norm r_now: {np.vdot(r_now, r_now).real}, Norm rr: {np.vdot(rr, rr).real}")

Erwartungswert des Hamiltonians: 0.021204543488659442
Erwartungswert des originalen Hamiltonians: -3.1653416760703355
Norm r_now: 7.365308161264373e-05, Norm rr: 0.9999999999198295


In [34]:
psi = r_now
rr2 = my_one_iter(time, n_steps, order, opt_angles)
r_now = rr2.reshape(2,-1)[0]

In [35]:
r_now = rr2.reshape(2,-1)[0]
energy = np.vdot(r_now, H_penny.sparse_matrix().toarray() @ r_now).real / np.vdot(r_now, r_now).real
print(f"Erwartungswert des Hamiltonians: {energy}")
energy = np.vdot(r_now, H_original @ r_now).real / np.vdot(r_now, r_now).real
print(f"Erwartungswert des originalen Hamiltonians: {energy}")
print(f"Norm r_now: {np.vdot(r_now, r_now).real}, Norm rr: {np.vdot(rr, rr).real}")

Erwartungswert des Hamiltonians: 0.02000000563772875
Erwartungswert des originalen Hamiltonians: -3.2077507447275844
Norm r_now: 9.891641280929664e-06, Norm rr: 0.9999999999198295


# The following should and does not work, as the of diagonal elements are not zero of cause

In [36]:
psi_full = rr2

@qml.qnode(dev)
def my_one_iter_full(time, n_steps=1, order=2, angles=None):
    # Prepare some state
    qml.StatePrep(psi_full, wires=range(grouped_H.num_wires + 1))
    iterate(angles)
        
    # Measure some quantity
    return qml.state()    